## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019A308A8430>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019A308A97E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish! It's nice to meet you. That's a fascinating title. \n\nWhat kind of projects are you working on as Chief AI Engineer? I'm always curious to learn about the cutting-edge applications of artificial intelligence.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_time': 0.096363636, 'prompt_time': 0.002117865, 'queue_time': 0.23287306900000002, 'total_time': 0.098481501}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cbacc653-de4f-49a3-89c5-3452454d1a49-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish and you're a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.005273349, 'queue_time': 0.23128458699999999, 'total_time': 0.088909713}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-85777068-4f8f-4a67-a64c-78a9f9823307-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}


In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Mahendhar and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

"Hello Mahendhar, it's nice to meet you!  \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  \n\nI'm curious to learn more about your work in the field of AI.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Mahendhar. 😊 \n\nI remembered from our earlier conversation!  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_time': 0.058181818, 'prompt_time': 0.005336699, 'queue_time': 0.235112386, 'total_time': 0.063518517}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-30ef0b2c-2acd-4d9f-8c6e-5964d3a1c629-0', usage_metadata={'input_tokens': 95, 'output_tokens': 32, 'total_tokens': 127})

In [ ]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content